# 文本预处理
文本是最常见的序列，文章是单词的序列或者是字符的序列。

文本的预处理常见步骤包括：
1. 将文本加载到内存中
2. 将文本拆分为单词或者字符
3. 建立一个词表，将拆分的单词或者字符映射到数字索引
4. 将文本转换为数字索引序列，方便模型操作

In [5]:
import torch as t
import torch.nn as nn
import sys
import re
import collections

sys.path.append("../")
from pltutils import *

In [6]:
def read_time_machine():
    with open(download("time_machine"),"r") as f:
        lines=f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f"总行数:{len(lines)}")
print(lines[0])
print(lines[10])

总行数:3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [7]:
#词元化

def tokenize(lines,token="word"):
    if token=="word":
        return [line.split() for line in lines]
    elif token=="char":
        return [list(line) for line in lines]
    else:
        raise NotImplementedError("only support Word and Char")

tokens=tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [8]:
# 词表
def count_corpus(tokens:list):
    # token 是1D列表或者是2D列表
    if len(tokens)==0 or isinstance(tokens[0],list):
        tokens=[token for line in tokens for token in line]
    return collections.Counter(tokens)
class Vocab:
    """
    文本词表
    """
    def __init__(self,tokens=None,min_freq=0,reserved_tokens=None):
        if tokens is None:
            tokens=[]
        if reserved_tokens is None:
            reserved_tokens=[]
        # 按出现的频率排序
        counter=count_corpus(tokens)
        self._token_freqs=sorted(counter.items(),key=lambda x:x[1],reverse=True)

        # 未知词元的索引为0
        self.idx_to_token=["<unk>"]+reserved_tokens
        self.token_to_idx={token:idx for idx,token  in enumerate(self.idx_to_token)}

        for token,freq in self._token_freqs:
            if freq<min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token]=len(self.idx_to_token)-1
    
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self,tokens):
        if not isinstance(tokens,(list,tuple)):
            return self.token_to_idx.get(tokens,self.unk)
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self,indices):
        if not isinstance(indices,(list,tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    
    @property
    def unk(self):
        return 0
    
    @property
    def token_freaqs(self):
        return self._token_freqs


In [10]:
vocab=Vocab(tokens=tokens)
print(list(vocab.token_to_idx.items())[:15])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9), ('my', 10), ('it', 11), ('had', 12), ('me', 13), ('as', 14)]


In [11]:
# 将文本转换为数字索引表
for i in [0,10]:
    print("text:",tokens[i])
    print("idx:",vocab[tokens[i]])

text: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
idx: [1, 19, 50, 40, 2183, 2184, 400]
text: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
idx: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [13]:
# 整合所有功能
def load_corpus_time_machine(max_tokens=-1):
    lines=read_time_machine()
    tokens=tokenize(lines,"char")
    vocab=Vocab(tokens)
    corpus=[vocab[token] for line in tokens for token in line]
    if max_tokens>0:
        corpus=corpus[:max_tokens]
    return corpus,vocab


corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)


(170580, 28)